In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


import time
from selenium.webdriver.common.keys import Keys

import numpy as np
import pandas as pd
import os


## Clean dataset

In [15]:
os.chdir('/Users/alisdghnia/Desktop/')

In [16]:
coin_list_2 = []
with open('coin_list.txt') as f:
    lines = f.readlines()
    for line in lines:
        coin_list_2.append(line.strip())
coin_list_2

['0chain - Whitepaper.pdf',
 '0x - Whitepaper.pdf',
 '0xbitcoin - Whitepaper.pdf',
 '0xcert - Whitepaper.pdf',
 '12Ships - Whitepaper.pdf',
 '1inch - Whitepaper.pdf',
 '1irstcoin - Whitepaper.pdf',
 '1World - Whitepaper.pdf',
 'Aave - Whitepaper.pdf',
 'AAX Token - Whitepaper.pdf',
 'ABBC Foundation - Whitepaper.pdf',
 'ABCC Token - Whitepaper.pdf',
 'Abjcoin - Whitepaper.pdf',
 'Able - Whitepaper.pdf',
 'Absolute - Whitepaper.pdf',
 'Abyss - Whitepaper.pdf',
 'AceD - Whitepaper.pdf',
 'Achain - Whitepaper.pdf',
 'Acoin - Whitepaper.pdf',
 'Acre - Whitepaper.pdf',
 'Actinium - Whitepaper.pdf',
 'Acute Angle Cloud - Whitepaper.pdf',
 'AdCoin - Whitepaper.pdf',
 'AdEx - Whitepaper.pdf',
 'Aditus - Whitepaper.pdf',
 'Adora - Whitepaper.pdf',
 'AdShares - Whitepaper.pdf',
 'adToken - Whitepaper.pdf',
 'Adzcoin - Whitepaper.pdf',
 'aelf - Whitepaper.pdf',
 'AENCoin - Whitepaper.pdf',
 'Aeon - Whitepaper.pdf',
 'Aergo - Whitepaper.pdf',
 'Aeron - Whitepaper.pdf',
 'Aeternity - Whitepaper.pdf

In [17]:
def DropWhitePaperSuffix(x):
    x = str(x).replace(' - Whitepaper.pdf','')
    return x

coin_list = [DropWhitePaperSuffix(x) for x in coin_list_2]
coin_list

['0chain',
 '0x',
 '0xbitcoin',
 '0xcert',
 '12Ships',
 '1inch',
 '1irstcoin',
 '1World',
 'Aave',
 'AAX Token',
 'ABBC Foundation',
 'ABCC Token',
 'Abjcoin',
 'Able',
 'Absolute',
 'Abyss',
 'AceD',
 'Achain',
 'Acoin',
 'Acre',
 'Actinium',
 'Acute Angle Cloud',
 'AdCoin',
 'AdEx',
 'Aditus',
 'Adora',
 'AdShares',
 'adToken',
 'Adzcoin',
 'aelf',
 'AENCoin',
 'Aeon',
 'Aergo',
 'Aeron',
 'Aeternity',
 'AgaveCoin',
 'Agrello',
 'Agrocoin',
 'Agrolot',
 'AI Chain',
 'AI Doctor',
 'AIChain',
 'AidCoin',
 'Aidos Kuneen',
 'Aidus',
 'AiLink Token',
 'Aion',
 'AIOZ Network',
 'Airbloc',
 'AirSwap',
 'AirToken',
 'Aitheon',
 'Akash Network',
 'Akroma',
 'Akropolis',
 'Aladdin',
 'ALAX',
 'Alchemint Standards',
 'Algorand',
 'Alibabacoin',
 'ALIS',
 'All Sports',
 'ALLY',
 'Alpha Coin',
 'Alpha Finance Lab',
 'Alpha Token',
 'Alphacat',
 'Alphacoin',
 'ALQO',
 'Amark',
 'Ambrosus',
 'Amino Network',
 'AMLT Token',
 'Amon',
 'Amond',
 'Ampleforth',
 'Anchor',
 'Anchor Neural World',
 'Ankr'

## Scrap the coin statistics

In [18]:
driver = webdriver.Chrome(executable_path='chromedriver.exe')

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_48124/1569032461.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver.exe')


In [19]:
def GoToCoinMarket():
    # Go to the page (coin market cap)
    search_result_links = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//div[@class="yuRUbf"]//a'))
            )
    for link in search_result_links:
        link = link.get_attribute('href')
        print(link)

        if ('coinmarketcap' in link):
            # Go to coin market if found
            driver.get(link)
            break

In [20]:
def GetCoinPrice():
    # get the coin price
    try:
        coin_price = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//div[@class="sc-aef7b723-0 dDQUel priceTitle"]//span')))
        coin_price = coin_price.text
    except:
        coin_price = 'Nan'

    # Get the market cap, dillute and volume
    try:
        coin_info = WebDriverWait(driver, 3).until(
        EC.presence_of_all_elements_located((By.XPATH, '//div[@class = "statsItemRight"]//div[@class="statsValue"]')))

        market_cap = coin_info[0].text
        fully_diluted_market_cap = coin_info[1].text
        volume = coin_info[2].text

    except:
        market_cap = 'NaN'
        fully_diluted_market_cap = 'NaN'
        volume = 'NaN'

    # Get max supply and total supply
    try:
        coin_info_2 = WebDriverWait(driver, 3).until(
        EC.presence_of_all_elements_located((By.XPATH, '//div[@class="maxSupplyValue"]')))

        max_supply = coin_info_2[0].text
        total_supply = coin_info_2[1].text 
    except:
        max_supply = 'NaN'
        total_supply = 'NaN'

    # get the coin name
    try:
        coin_name_on_web = driver.find_element(By.XPATH,'//h2[@class="sc-aba8b85a-0 gmYubB h1"]//span//span')
        coin_name_on_web = coin_name_on_web.text
    except:
        coin_name_on_web = 'NaN'

    # get coin symbol
    try:
        coin_symbol = driver.find_element(By.XPATH, '//small[@class="nameSymbol"]')
        coin_symbol = coin_symbol.text
    except:

        coin_symbol = 'NaN'

    return [coin_price,market_cap,volume, max_supply,total_supply,fully_diluted_market_cap, coin_name_on_web, coin_symbol]

In [21]:
def SearchCoin(topic_search):

    try: 
        topic_search = topic_search.replace(' ', '+')

        for i in range(1):
            search_phrase = 'https://www.google.com/search?q='+ topic_search + '&start' + str(i)
            driver.get(search_phrase)
            
            # sleep a little
            time.sleep(1)
            GoToCoinMarket()
            coin = GetCoinPrice()
            return coin
            
    except:
        return ['Nan','Nan','Nan','Nan','Nan','Nan']
    


In [22]:
coin_price = pd.DataFrame(columns=['coin_whitepaper_name', 'coin_name_on_web', 'coin_symbol','coin_price','market_cap', 'volume', 'max_supply', 'total_supply', 'fully_diluted_market_cap'])

In [23]:
for i in range(840,920):

    topic_search = str(coin_list[i]) + (' coin coinmarketcap')
    coin = SearchCoin(topic_search=topic_search)

    coin_price = coin_price.append({'coin_whitepaper_name': str(coin_list_2[i]),'coin_name_on_web' : coin[6], 'coin_symbol':coin[7]  ,'coin_price': coin[0],
                                    'market_cap':coin[1], 'volume':coin[2], 'max_supply':coin[3], 
                                    'total_supply':coin[4], 'fully_diluted_market_cap':coin[5]}, ignore_index=True)
    
coin_price

https://coinmarketcap.com/currencies/innovaminex/


/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_48124/2398421810.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_price = coin_price.append({'coin_whitepaper_name': str(coin_list_2[i]),'coin_name_on_web' : coin[6], 'coin_symbol':coin[7]  ,'coin_price': coin[0],


https://coinmarketcap.com/currencies/innovative-bioresearch-classic/


/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_48124/2398421810.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_price = coin_price.append({'coin_whitepaper_name': str(coin_list_2[i]),'coin_name_on_web' : coin[6], 'coin_symbol':coin[7]  ,'coin_price': coin[0],


https://coinmarketcap.com/currencies/ino-coin/


...

https://coinmarketcap.com/currencies/landshare/


/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_48124/2398421810.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_price = coin_price.append({'coin_whitepaper_name': str(coin_list_2[i]),'coin_name_on_web' : coin[6], 'coin_symbol':coin[7]  ,'coin_price': coin[0],


https://coinmarketcap.com/currencies/latium-old/


/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_48124/2398421810.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_price = coin_price.append({'coin_whitepaper_name': str(coin_list_2[i]),'coin_name_on_web' : coin[6], 'coin_symbol':coin[7]  ,'coin_price': coin[0],


,coin_whitepaper_name,coin_name_on_web,coin_symbol,coin_price,market_cap,volume,max_supply,total_supply,fully_diluted_market_cap
0,Innovaminex - Whitepaper.pdf,InnovaMinex,MINX,Nan,NaN,NaN,NaN,NaN,NaN
1,Innovative Bioresearch Classic - Whitepaper.pdf,Innovative Bioresearch Classic,INNBCL,$0.0...09989,$0,- -,"5,000,000,000","5,000,000,000",$0
2,INO Coin - Whitepaper.pdf,Ino Coin,INO,$0.009991,"$1,798,367",- -,--,"1,000,000,000","$9,990,753"
3,InPay - Whitepaper.pdf,Pay It Now,PIN,$0.01066,- -,"$21,238","100,000,000","100,000,000","$1,066,441"
4,INS Ecosystem - Whitepaper.pdf,Ecosystem Coin Network,ECN,$0.5195,- -,- -,"270,000,000","270,000,000","$140,269,641"
...,...,...,...,...,...,...,...,...,...
75,Lala World - Whitepaper.pdf,LALA World,LALA,Nan,NaN,NaN,NaN,NaN,NaN
76,Lambda - Whitepaper .pdf,Lambda,LAMB,$0.001125,"$1,852,717","$341,304","10,000,000,000","6,000,000,000","$11,253,044"
77,Lamden - Whitepaper.pdf,Lamden,TAU,$0.01363,"$1,938,585","$12,818","288,090,567","248,090,567","$3,927,048"
78,Landshare - Whitepaper.pdf,Landshare,LAND,$0.5417,"$1,673,430","$45,651","10,000,000","3,823,303","$5,416,833"


In [24]:
driver.quit()

In [25]:
coin_price

,coin_whitepaper_name,coin_name_on_web,coin_symbol,coin_price,market_cap,volume,max_supply,total_supply,fully_diluted_market_cap
0,Innovaminex - Whitepaper.pdf,InnovaMinex,MINX,Nan,NaN,NaN,NaN,NaN,NaN
1,Innovative Bioresearch Classic - Whitepaper.pdf,Innovative Bioresearch Classic,INNBCL,$0.0...09989,$0,- -,"5,000,000,000","5,000,000,000",$0
2,INO Coin - Whitepaper.pdf,Ino Coin,INO,$0.009991,"$1,798,367",- -,--,"1,000,000,000","$9,990,753"
3,InPay - Whitepaper.pdf,Pay It Now,PIN,$0.01066,- -,"$21,238","100,000,000","100,000,000","$1,066,441"
4,INS Ecosystem - Whitepaper.pdf,Ecosystem Coin Network,ECN,$0.5195,- -,- -,"270,000,000","270,000,000","$140,269,641"
...,...,...,...,...,...,...,...,...,...
75,Lala World - Whitepaper.pdf,LALA World,LALA,Nan,NaN,NaN,NaN,NaN,NaN
76,Lambda - Whitepaper .pdf,Lambda,LAMB,$0.001125,"$1,852,717","$341,304","10,000,000,000","6,000,000,000","$11,253,044"
77,Lamden - Whitepaper.pdf,Lamden,TAU,$0.01363,"$1,938,585","$12,818","288,090,567","248,090,567","$3,927,048"
78,Landshare - Whitepaper.pdf,Landshare,LAND,$0.5417,"$1,673,430","$45,651","10,000,000","3,823,303","$5,416,833"


In [26]:
coin_price.to_csv('Coin-Price-Match 840-919.csv')